Import Library

In [260]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import itertools 

In this notebook, I want to analyze who is the best player in vct 2024 and how the best team will look like. At this point, I have no idea on what to do. But we will see.

In [261]:
player_data = pd.read_csv("dataset/vct_2024/players_stats/players_stats.csv")
player_data.head()

,Tournament,Stage,Match Type,Player,Teams,Agents,Rounds Played,Rating,Average Combat Score,Kills:Deaths,...,First Deaths Per Round,Headshot %,Clutch Success %,Clutches (won/played),Maximum Kills in a Single Map,Kills,Deaths,Assists,First Kills,First Deaths
0,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,astra,26,0.75,138,0.58,...,0.08,21%,22%,2/9,11,11,19,14,1,2
1,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,fade,16,1.10,185,1.38,...,0.00,21%,NaN,NaN,11,11,8,3,2,0
2,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,"astra, fade",42,0.88,162,0.81,...,0.05,21%,22%,2/9,11,22,27,17,3,2
3,Valorant Champions 2024,Playoffs,Upper Quarterfinals,benjyfishy,Team Heretics,cypher,26,0.85,203,0.86,...,0.15,34%,NaN,NaN,18,18,21,3,2,4
4,Valorant Champions 2024,Playoffs,Upper Quarterfinals,benjyfishy,Team Heretics,killjoy,16,1.36,169,2.40,...,0.00,28%,50%,1/2,12,12,5,2,1,0


In [262]:
player_data.shape

(15030, 25)

In valorant, a team of 5 is usually consist of 1 duelist, 1 controller, 1 sentinel, 1 initiator, and 1 flex (can play most role if not all) player. Right now, I will not prioritize IGL within a team since I think it will make the code harder to do (and also because PRX can still play good without proper IGL). 

I will choose the best player for each role and combine them to make the best team. For someone to be considered as a duelist/sentinel/controller/initiator, they need to play the role for about 70 % of the time. For flex role, they should be able to play more than 10 agent to be considered as a flex. 

In [263]:
agent_dictionary = { 
    "sentinel": ["sage", "cypher", "killjoy", "chamber", "deadlock", "vyse"],
    "controller": ["brimstone", "viper", "omen", "astra", "harbor", "clove"],
    "duelist": ["jett", "reyna", "phoenix", "raze", "yoru", "neon", "iso"],
    "initiator": ["sova", "breach", "skye", "kayo", "fade", "gekko", "tejo"]
}

inverted_agent_dictionary = {
  'sage': 'sentinel',
  'cypher': 'sentinel',
  'killjoy': 'sentinel',
  'chamber': 'sentinel',
  'deadlock': 'sentinel',
  'vyse': 'sentinel',
  'brimstone': 'controller',
  'viper': 'controller',
  'omen': 'controller',
  'astra': 'controller',
  'harbor': 'controller',
  'clove': 'controller',
  'jett': 'duelist',
  'reyna': 'duelist',
  'phoenix': 'duelist',
  'raze': 'duelist',
  'yoru': 'duelist',
  'neon': 'duelist',
  'iso': 'duelist',
  'sova': 'initiator',
  'breach': 'initiator',
  'skye': 'initiator',
  'kayo': 'initiator',
  'fade': 'initiator',
  'gekko': 'initiator',
  'tejo': 'initiator'
}

# flatten list
all_agents = list(itertools.chain.from_iterable([agent for agent in agent_dictionary.values()]))
all_agents

['sage',
 'cypher',
 'killjoy',
 'chamber',
 'deadlock',
 'vyse',
 'brimstone',
 'viper',
 'omen',
 'astra',
 'harbor',
 'clove',
 'jett',
 'reyna',
 'phoenix',
 'raze',
 'yoru',
 'neon',
 'iso',
 'sova',
 'breach',
 'skye',
 'kayo',
 'fade',
 'gekko',
 'tejo']

as you can see in the bottom, for one match, there are a "compiled" version of the game. For example, index 0 is game 1 and index 1 is game 2, meanwhile index 2 is the combined version of game 1 and game 2. For the sake of simplicity, I will just leave the "compiled" version and just use the single version. 

In [264]:
player_data.head(3)

,Tournament,Stage,Match Type,Player,Teams,Agents,Rounds Played,Rating,Average Combat Score,Kills:Deaths,...,First Deaths Per Round,Headshot %,Clutch Success %,Clutches (won/played),Maximum Kills in a Single Map,Kills,Deaths,Assists,First Kills,First Deaths
0,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,astra,26,0.75,138,0.58,...,0.08,21%,22%,2/9,11,11,19,14,1,2
1,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,fade,16,1.10,185,1.38,...,0.00,21%,NaN,NaN,11,11,8,3,2,0
2,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,"astra, fade",42,0.88,162,0.81,...,0.05,21%,22%,2/9,11,22,27,17,3,2


In [265]:
player_data = player_data[~player_data["Agents"].str.contains(",")]

i do a little bit of preprocessing because several columns should be in numerical, not object

In [266]:
player_data["Kill, Assist, Trade, Survive %"] = player_data["Kill, Assist, Trade, Survive %"].str.replace("%", "").astype(float)
player_data["Headshot %"] = player_data["Headshot %"].str.replace("%", "").astype(float)
player_data["Clutch Success %"] = player_data["Clutch Success %"].str.replace("%", "").astype(float)
player_data["Clutch Success %"] = player_data["Clutch Success %"].fillna(0)
player_data[['Clutch won', 'Clutch played']] = player_data['Clutches (won/played)'].str.split('/', expand=True)
player_data[['Clutch won', 'Clutch played']] = player_data[['Clutch won', 'Clutch played']].fillna(0)
player_data['Clutch won'] = player_data['Clutch won'].astype(int)
player_data['Clutch played'] = player_data['Clutch played'].astype(int)
player_data = player_data.drop(columns="Clutches (won/played)")

In [267]:
tournament_list = player_data["Tournament"].unique().tolist()

In [268]:
cat_cols = ["Tournament"]
player_data = pd.get_dummies(player_data, columns = cat_cols)
player_data = player_data.drop(columns=["Match Type", "Stage", "Teams"])

In [269]:
player_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10969 entries, 0 to 15028
Data columns (total 37 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Player                                            10969 non-null  object 
 1   Agents                                            10969 non-null  object 
 2   Rounds Played                                     10969 non-null  int64  
 3   Rating                                            8057 non-null   float64
 4   Average Combat Score                              10969 non-null  int64  
 5   Kills:Deaths                                      10969 non-null  float64
 6   Kill, Assist, Trade, Survive %                    8057 non-null   float64
 7   Average Damage Per Round                          9519 non-null   float64
 8   Kills Per Round                                   10969 non-null  float64
 9   Assists Per Round     

next, I want to check what are the agents played since there are new agents in 2025

In [270]:
for agent in all_agents:
    if agent not in player_data["Agents"].unique().tolist():
        print(f"this agent {agent} is not on the list")

this agent vyse is not on the list
this agent tejo is not on the list


vyse was released in august 2024 (after champions 2024) and tejo was released in january 2025. This shows that this two agent were not available back then in vct 2024 season.

In [271]:
player_data["Player"].nunique() # there are 260 players

260

Next, since I'm determining who is the best player for each role in 2024, I think I will change the table a bit so that the player will be the main variable. From the code above, we can also know that there are 260 players in total.

I'm planning to group each player and get several important data from that and joined it all later in one dataframe like this. 

example : 
                
    player name      tournament played   regional winner     international winner    acs     kd      hs%       rating

    f0rsaken           20                   3                   0               500     3.4     50          1.76

    woot               100                  4                   0               400     2.4     24          1.4

    d4v4i              30                   5                   0               300     1.2     35          1.2

Next, I want to investigate one player first

In [272]:
df = player_data.copy() 

data = df.groupby("Player")
data.get_group("f0rsakeN")

,Player,Agents,Rounds Played,Rating,Average Combat Score,Kills:Deaths,"Kill, Assist, Trade, Survive %",Average Damage Per Round,Kills Per Round,Assists Per Round,...,Tournament_Champions Tour 2024: China Stage 2,Tournament_Champions Tour 2024: EMEA Kickoff,Tournament_Champions Tour 2024: EMEA Stage 1,Tournament_Champions Tour 2024: EMEA Stage 2,Tournament_Champions Tour 2024: Masters Madrid,Tournament_Champions Tour 2024: Masters Shanghai,Tournament_Champions Tour 2024: Pacific Kickoff,Tournament_Champions Tour 2024: Pacific Stage 1,Tournament_Champions Tour 2024: Pacific Stage 2,Tournament_Valorant Champions 2024
859,f0rsakeN,kayo,22,1.32,239,1.12,77.0,146.0,0.86,0.64,...,False,False,False,False,False,False,False,False,False,True
860,f0rsakeN,sova,21,0.85,182,0.82,62.0,123.0,0.67,0.24,...,False,False,False,False,False,False,False,False,False,True
917,f0rsakeN,breach,24,0.70,149,0.61,79.0,95.0,0.46,0.54,...,False,False,False,False,False,False,False,False,False,True
918,f0rsakeN,fade,19,1.11,157,1.63,84.0,113.0,0.68,0.32,...,False,False,False,False,False,False,False,False,False,True
945,f0rsakeN,breach,21,0.17,65,0.26,43.0,43.0,0.24,0.00,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14262,f0rsakeN,cypher,47,0.75,170,0.79,64.0,114.0,0.57,0.21,...,False,False,False,False,False,False,True,False,False,False
14263,f0rsakeN,fade,37,1.11,205,1.15,78.0,110.0,0.81,0.41,...,False,False,False,False,False,False,True,False,False,False
14264,f0rsakeN,harbor,23,0.89,203,0.94,78.0,142.0,0.74,0.35,...,False,False,False,False,False,False,True,False,False,False
14265,f0rsakeN,kayo,20,1.44,309,1.50,85.0,196.0,1.05,0.40,...,False,False,False,False,False,False,True,False,False,False


from code below, i was able to gain some valuable information. now, i want to generalize it and make a function instead

In [290]:
# forsaken_data = data.get_group("f0rsakeN")

# col = ["Rating", "Average Combat Score", "Kills:Deaths", "Kill, Assist, Trade, Survive %", "Average Damage Per Round",
#        "Kills Per Round", "Assists Per Round", "First Kills Per Round", "First Deaths Per Round", "Headshot %", "Clutch Success %"]

# for c in col:
#     forsaken_data[c] = forsaken_data["Rounds Played"] * forsaken_data[c]
# new_data = forsaken_data.sum().copy()
# rd = new_data["Rounds Played"]
# new_data[col] = new_data[col]/rd
# new_data = new_data.drop(labels=["Player", "Agents"])

# new_data["Maximum Kills in a Single Map"] = forsaken_data["Maximum Kills in a Single Map"].max()
# new_data["number of agents played"] = forsaken_data["Agents"].nunique()


# dic = data.get_group("f0rsakeN")["Agents"].value_counts().to_dict()
# dic

# roles = {"sentinel":0, "controller":0, "initiator":0, "duelist":0}

# for key in dic:
#     agent_role = inverted_agent_dictionary[key]
#     roles[agent_role] += dic[key]

# # Sum of all counts
# total = sum(roles.values())

# # Convert each count to a percentage
# percentages = {role: (count / total) * 100 for role, count in roles.items()}

# # Optionally, round the percentages
# percentages = {role: round(pct, 2) for role, pct in percentages.items()}

# for p in percentages:
#     new_data[p] = percentages[p]

# for tour in tournament_list:
#     new_data[f"Tournament_{tour}"] = new_data[f"Tournament_{tour}"] > 0

# new_data

AttributeError: 'Series' object has no attribute 'get_group'

In [299]:
def get_data_from_player(df, player_name):
    data = df.groupby("Player")
    p_data = data.get_group(player_name)

    col = ["Rating", "Average Combat Score", "Kills:Deaths", "Kill, Assist, Trade, Survive %", "Average Damage Per Round",
       "Kills Per Round", "Assists Per Round", "First Kills Per Round", "First Deaths Per Round", "Headshot %", "Clutch Success %"]

    for c in col:
        p_data[c] = p_data["Rounds Played"] * p_data[c]
    
    new_data = p_data.sum().copy()
    rd = new_data["Rounds Played"]
    new_data[col] = new_data[col]/rd
    new_data = new_data.drop(labels=["Player", "Agents"])

    dic = data.get_group(player_name)["Agents"].value_counts().to_dict()
    dic

    roles = {"sentinel":0, "controller":0, "initiator":0, "duelist":0}

    for key in dic:
        agent_role = inverted_agent_dictionary[key]
        roles[agent_role] += dic[key]

    # Sum of all counts
    total = sum(roles.values())

    # Convert each count to a percentage
    percentages = {role: (count / total) * 100 for role, count in roles.items()}

    # Optionally, round the percentages
    percentages = {role: round(pct, 2) for role, pct in percentages.items()}

    for p in percentages:
        new_data[p] = percentages[p]

    for tour in tournament_list:
        new_data[f"Tournament_{tour}"] = new_data[f"Tournament_{tour}"] > 0

    new_data["Player"] = player_name

    return new_data


In [304]:
player_list = player_data["Player"].unique().tolist()
series_list = []

for player in player_list:
    player_vct_data = get_data_from_player(player_data, player)
    series_list.append(player_vct_data)

dataframe_player = pd.concat(series_list, axis = 1).T

cols = dataframe_player.columns.tolist()
dataframe_player = dataframe_player[[cols[-1]] + cols[:-1]]
dataframe_player

/var/folders/c1/g_qgswp92pn07w79flqmp52m0000gn/T/ipykernel_46772/2180038880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_data[c] = p_data["Rounds Played"] * p_data[c]
/var/folders/c1/g_qgswp92pn07w79flqmp52m0000gn/T/ipykernel_46772/2180038880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_data[c] = p_data["Rounds Played"] * p_data[c]
/var/folders/c1/g_qgswp92pn07w79flqmp52m0000gn/T/ipykernel_46772/2180038880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

,Player,Rounds Played,Rating,Average Combat Score,Kills:Deaths,"Kill, Assist, Trade, Survive %",Average Damage Per Round,Kills Per Round,Assists Per Round,First Kills Per Round,...,Tournament_Champions Tour 2024: Masters Madrid,Tournament_Champions Tour 2024: Masters Shanghai,Tournament_Champions Tour 2024: Pacific Kickoff,Tournament_Champions Tour 2024: Pacific Stage 1,Tournament_Champions Tour 2024: Pacific Stage 2,Tournament_Valorant Champions 2024,sentinel,controller,initiator,duelist
0,Boo,4734,0.770165,173.36185,0.984278,58.041191,111.285594,0.620059,0.409463,0.080355,...,True,True,False,False,False,True,4.9,88.24,6.86,0.0
1,benjyfishy,4734,0.832256,198.49831,1.154472,56.256654,132.659273,0.737607,0.186538,0.088754,...,True,True,False,False,False,True,76.34,23.66,0.0,0.0
2,Wo0t,3738,0.778419,215.038523,1.103424,55.599786,140.73435,0.748601,0.318874,0.135037,...,False,True,False,False,False,True,0.0,16.3,56.52,27.17
3,MiniBoo,3750,1.013904,220.162133,1.050776,71.4352,144.234133,0.765011,0.218616,0.174176,...,True,False,False,False,False,True,0.0,0.0,7.14,92.86
4,RieNs,4734,0.865256,205.85319,1.119438,59.834601,138.411914,0.722655,0.354052,0.077022,...,True,True,False,False,False,True,0.0,6.67,93.33,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,jakee,324,0.964136,171.191358,0.897191,73.975309,113.32716,0.60429,0.3875,0.04321,...,False,False,False,False,False,False,0.0,100.0,0.0,0.0
256,wippie,324,0.989877,179.469136,1.015494,73.549383,119.734568,0.605802,0.223272,0.066914,...,False,False,False,False,False,False,50.0,50.0,0.0,0.0
257,Egoist,244,1.062336,202.147541,1.108607,75.418033,132.008197,0.745861,0.228402,0.084713,...,False,False,True,False,False,False,71.43,0.0,28.57,0.0
258,Destrian,476,0.842374,156.85084,0.789916,68.586134,112.008403,0.537521,0.319055,0.058718,...,False,False,False,False,False,False,57.14,42.86,0.0,0.0


now, i think i finish the preprocessing. time to analyze the data